In [ ]:
"""
Created on Sat Oct 18 11:08:38 2021
@author: Mayank
Rollno:B20156
PhoneNo: +919636993445
"""

In [ ]:
#import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.mixture import GaussianMixture
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score 
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import statistics as st
#filter warning
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#read csv
df=pd.read_csv('abalone.csv')
#list of columns
col=list(df.columns)
#split test and train data
[df_train, df_test] = train_test_split(df, test_size=0.3, random_state=42, shuffle=True)

In [ ]:
#copy of test and train data
df1_train=df_train.copy(deep=True)
df1_test=df_test.copy(deep=True)

PearCoff=[]
#Calculate Pearson Coefficient
for x in col:
    if x!="Rings":
        col1=df1_train[x]
        col2=df1_train["Rings"]
        PearCoff.append(col1.corr(col2))
#element with max pearson coefficient
Maxcorrel=col[PearCoff.index(max(PearCoff))]
print("Attribute with Max Pearson Correlation :",Maxcorrel)

#coefficient of Shell weight
coffx=(df1_train[Maxcorrel].cov(df1_train['Rings']))/(df1_train[Maxcorrel].cov(df1_train[Maxcorrel]))
#Constant
cons=df1_train['Rings'].mean(axis = 0)-(df1_train[Maxcorrel].mean(axis = 0)*coffx)

#Q1a
#plot best fit line
X=np.linspace(0, 1.0, num=5)
Y=coffx*X+cons
plt.plot(X,Y, label ="Best Fit Line",color='r')
#title
plt.title("Best Fit Line and Training Data")
#labels
plt.xlabel("Shell Weight")
plt.ylabel("Rings")
#scatter data
plt.scatter(df1_train[Maxcorrel],df1_train['Rings'])
#plot grid
plt.grid()
#show plot
plt.show()

#Q1b
#find RMSE of train data for each tuple
trainerror=0
for x in range(0,len(df1_train)):
    trainerror+=(((df1_train[Maxcorrel].iloc[x]*coffx)+cons-df1_train['Rings'].iloc[x])**2)/len(df1_train)
#print result
print("Root mean squared error in train data:",trainerror**0.5)

#Q1c
testerror=0
#find RMSE of test data for each tuple
for x in range(0,len(df1_test)):
    testerror+=(((df1_test[Maxcorrel].iloc[x]*coffx)+cons-df1_test['Rings'].iloc[x])**2)/len(df1_test)
#print result
print("Root mean squared error in test data:",testerror**0.5)

#Q1d
#predicted value as Rings using Shell weights
Ypred=coffx*df1_test[Maxcorrel]+cons
#scatter data
plt.scatter(df1_test['Rings'],Ypred,color='g')
#title and labels
plt.title("Actual Values of Rings and their Predicted Values")
plt.xlabel("Actual Value of Rings")
plt.ylabel("Predicted Value of Rings")
#show grid
plt.grid()
#show plot
plt.show()

In [ ]:
#copy of train and test data
df2_train=df_train.copy(deep=True)
df2_test=df_test.copy(deep=True)

#dataframe of train rings
trainRings=df2_train['Rings']
#drop train rings
df2_train.drop(columns=['Rings'],inplace=True)
#dataframe of test rings
testRings=df2_test['Rings']
#drop test rings
df2_test.drop(columns=['Rings'],inplace=True)

#build multivariate linear regression model and fit train data
model = LinearRegression()
model.fit(df2_train,trainRings)

#2a
#predict train data
y_train_pred = model.predict(df2_train)
#find RMSE in train data and print
Trainerror=(trainRings-y_train_pred)**2
print("RMSE in multivariate model(Train data):",(sum(Trainerror)/len(Trainerror))**0.5)

#2b
#predict test data
y_test_pred = model.predict(df2_test)
#find RMSE in test data and print
Testerror=(testRings-y_test_pred)**2
print("RMSE in multivariate model(Test data):",(sum(Testerror)/len(Testerror))**0.5)

#Q2c
#scatter actual and predict value of test data Rings
plt.scatter(testRings,y_test_pred,color='g')
#title and labels
plt.title("Plot of Actual Values of Rings and their Predicted Values(Test Data)")
plt.xlabel("Actual Value(Rings)")
plt.ylabel("Predicted Value(Rings)")
#show grid
plt.grid()
#show plot
plt.show()


In [ ]:
#import train and test data
df3_train=df_train.copy(deep=True)
df3_test=df_test.copy(deep=True)
#input values for train and test data
Trin=np.array(df3_train[Maxcorrel])
Tein=np.array(df3_test[Maxcorrel])

#error list for train and  test data
errtrain=[]
errtest=[]
l=[2,3,4,5]

for x in (2,3,4,5):
    #fit input train and test data in a polynomial form with degree x
    poly_exp_train= PolynomialFeatures(degree=x, include_bias=True).fit_transform(Trin.reshape(-1,1))
    poly_exp_test= PolynomialFeatures(degree=x, include_bias=True).fit_transform(Tein.reshape(-1,1))
    #create model using train data
    model = LinearRegression(fit_intercept=True).fit(poly_exp_train, df3_train['Rings'])
    #predict output of train and test data Rings using model
    y_pred_train = model.predict(poly_exp_train)
    y_pred_test=model.predict(poly_exp_test)
    #calculate error for each model on training data
    sumtr=0
    for i in range(0,len(y_pred_train)):
        sumtr+=(((y_pred_train[i]-list(df3_train['Rings'])[i])**2)/len(y_pred_train))
    #append the error in list
    errtrain.append(sumtr**0.5)
    #calculate error for each model on testing data
    sumts=0
    for i in range(0,len(y_pred_test)):
        sumts+=(((y_pred_test[i]-list(df3_test['Rings'])[i])**2)/len(y_pred_test))
    #append the error in list
    errtest.append(sumts**0.5)

#print results
print("RMSE in trainig data:",errtrain)
print("RMSE in testing data:",errtest)

#3a
#plot bar graph of error in each model with x axis as degree
plt.bar(l,errtrain)
#title and labels
plt.title("RMSE in training data")
plt.xlabel("Degree of curve")
plt.ylabel("RMSE")
#show plot
plt.show()

#3b
#plot bar graph of error in each model with x axis as degree
plt.bar(l,errtest)
#title and labels
plt.xlabel("Degree of Curve")
plt.ylabel("RMSE")
plt.title("RMSE in testing dat")
#show plot
plt.show()

#3c
#input range
k=np.linspace(0,1,1000)
#create model using training data
poly_exp_train= PolynomialFeatures(degree=5, include_bias=True).fit_transform(Trin.reshape(-1,1))
poly_exp_k= PolynomialFeatures(degree=5, include_bias=True).fit_transform(k.reshape(-1,1))
model = LinearRegression(fit_intercept=True).fit(poly_exp_train, df3_train['Rings'])
#predict output using input line
y_pred_k = model.predict(poly_exp_k)
#plot line
plt.plot(k,y_pred_k,color='r')
plt.scatter(df3_train[Maxcorrel],df3_train['Rings'])
#labels and title
plt.title("Best Fit Curve")
plt.xlabel("Shell Weight")
plt.ylabel("Rings")
#show plot
plt.show()

#3d
#create model
poly_exp_train= PolynomialFeatures(degree=5, include_bias=True).fit_transform(Trin.reshape(-1,1))
poly_exp_test= PolynomialFeatures(degree=5, include_bias=True).fit_transform(Tein.reshape(-1,1))
model = LinearRegression(fit_intercept=True).fit(poly_exp_train, df3_train['Rings'])
#predict test data
y_pred_test=model.predict(poly_exp_test)
#scatter actual and predicted test Rings
plt.scatter(df3_test['Rings'],y_pred_test)
#labels and title
plt.title("Plot of Actual Values of Rings and their Predicted Values(Test Data) DEG=5")
plt.xlabel("Actual Value(Rings)")
plt.ylabel("Predicted Value(Rings)")
#show plot
plt.show()

In [ ]:
#copy dataframes
df4_train=df_train.copy(deep=True)
df4_test=df_test.copy(deep=True)
#create separate dataframes for actual output and drop those columns
y_train=np.array(df4_train['Rings'])
df4_train.drop(columns=['Rings'],inplace=True)
y_test=np.array(df4_test['Rings'])
df4_test.drop(columns=['Rings'],inplace=True)
l=[2,3,4,5]
trerror=[]
tserror=[]
for i in(2,3,4,5):
    #create model
    x_train = PolynomialFeatures(degree=i, include_bias=True).fit_transform(np.array(df4_train))
    x_test = PolynomialFeatures(degree=i, include_bias=True).fit_transform(np.array(df4_test))
    model = LinearRegression().fit(x_train, y_train)
    #predict test and train data
    y_pred_train = model.predict(x_train)
    y_pred_test = model.predict(x_test)
    #calculate and append train data error in list
    sumtr=0
    for i in range(0,len(y_pred_train)):
        sumtr+=(((y_pred_train[i]-y_train[i])**2)/len(y_pred_train))
    trerror.append(sumtr**0.5)
    #calculate and append test data error in list
    sumts=0
    for i in range(0,len(y_pred_test)):
        sumts+=(((y_pred_test[i]-y_test[i])**2)/len(y_pred_test))
    tserror.append(sumts**0.5)

#4a
#plot bar graph of training data error
plt.bar(l,trerror)
#labels and title
plt.title("RMSE in training data")
plt.xlabel("Degree of curve")
plt.ylabel("RMSE")
#show plot
plt.show()

#4b
#plot bar graph of testing data error
plt.bar(l,tserror)
#labels and title
plt.xlabel("Degree of Curve")
plt.ylabel("RMSE")
plt.title("RMSE in testing data")
#show plot
plt.show()

#4c
#build model and predict values for testing data
x_train = PolynomialFeatures(degree=2, include_bias=False).fit_transform(np.array(df4_train))
x_test = PolynomialFeatures(degree=2, include_bias=False).fit_transform(np.array(df4_test))
model = LinearRegression().fit(x_train, y_train)
y_pred_test = model.predict(x_test)
#scatter predicted and actual values of testing data output
plt.scatter(y_test,y_pred_test)
#labels and title
plt.title("Plot of Actual Values of Rings and their Predicted Values(Test Data) DEG=2")
plt.xlabel("Actual Value(Rings)")
plt.ylabel("Predicted Value(Rings)")
#show plot
plt.show()